In [1]:
######################
#### Dependencies ####
######################
import os 
import numpy as np
import pandas as pd
import datetime

def get_one_forecast(DAY,HOUR,main_dir='../data/raw/forecasts/'):
    forecast_dir_path = main_dir + DAY + '_' + HOUR + '/'
    # Init output
    data = pd.DataFrame()
    # Read 2 meteo file
    for dir_day in os.listdir(forecast_dir_path):
        if len(dir_day) == 10:
            # load file
            data_per_day = pd.read_csv(forecast_dir_path + dir_day + '/meteo.txt',delimiter=";")
            data_per_day['date'] = dir_day.replace('_','-')
            # append to output
            data = data.append(data_per_day)
            # add 1 day to fperiod

    # forecast date
    data['f_date'] = data['date'] + ' ' 
    data['f_date'] += data['heure'].map(lambda x : str(x).zfill(2))
    data['f_date'] += ':00:00'
    # present date
    data['p_date'] = DAY.replace('_','-') + ' ' + HOUR + ':00:00'

    # rename columns 
    data = data.rename(columns={'vitesse' : 'speed', 
                                'temperature' : 'temp', 
                                'rayonnement' : 'rad',
                                'direction' : 'wind_dir'})

    # compute cos and sin
    data['cos_wind_dir'] = np.cos(2 * np.pi * data['wind_dir'] / 360)
    data['sin_wind_dir'] = np.sin(2 * np.pi * data['wind_dir'] / 360)

    # select columns
    data = data[['p_date','f_date','speed','temp','rad','precip','cos_wind_dir','sin_wind_dir','wind_dir']]
    return data

# Define dir and tree
main_dir = '../data/raw/forecasts/'
sub_dir = os.listdir(main_dir)

# Date range to fetch : 5 months
end_date = pd.to_datetime(datetime.datetime.now())
start_date = pd.to_datetime(datetime.datetime.now() - datetime.timedelta(days=150))

# Concat files
forecast = pd.DataFrame()
for file_path in sub_dir:
    DAY = file_path[0:10]
    HOUR = file_path[11:13]
    file_path_date = pd.to_datetime(DAY.replace('_','-') + ' ' + HOUR + ':00:00')
    if (start_date <= file_path_date) & (file_path_date + datetime.timedelta(hours=8) <= end_date):
        forecast_part = get_one_forecast(DAY,HOUR)
        forecast_part['file_creation_date'] = file_path_date + datetime.timedelta(hours=8)
        forecast = forecast.append(forecast_part)
        del forecast_part
forecast = forecast.reset_index(drop=True)

# Change to dt 
forecast['f_date']= pd.to_datetime(forecast['f_date'],format='%Y-%m-%d %H:%M:%S')
forecast['p_date']= pd.to_datetime(forecast['p_date'],format='%Y-%m-%d %H:%M:%S')

# Save file
forecast.to_csv('../data/processed/last_forecast.csv',index=False)

forecast.head()

,p_date,f_date,speed,temp,rad,precip,cos_wind_dir,sin_wind_dir,wind_dir,file_creation_date
0,2021-03-04 12:00:00,2021-03-04 12:00:00,3.9,17.9,690,0.0,-0.438371,-0.898794,244,2021-03-04 20:00:00
1,2021-03-04 12:00:00,2021-03-04 13:00:00,4.3,17.6,736,0.5,-0.173648,-0.984808,260,2021-03-04 20:00:00
2,2021-03-04 12:00:00,2021-03-04 14:00:00,4.1,16.8,726,0.3,-0.121869,-0.992546,263,2021-03-04 20:00:00
3,2021-03-04 12:00:00,2021-03-04 15:00:00,3.8,16.6,660,0.2,0.139173,-0.990268,278,2021-03-04 20:00:00
4,2021-03-04 12:00:00,2021-03-04 16:00:00,3.3,16.1,543,0.2,0.374607,-0.927184,292,2021-03-04 20:00:00
